In [1]:
# Import des différents packages

import networkx as nx
import random
from collections import defaultdict
from typing import Dict, List
import matplotlib
import json
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Enregistrer les caractéristiques dans stats_twitter.json

def save_stats(stats):
    with open("stats_twitter.json", "w") as f:
        json.dump(stats, f, sort_keys=True, indent=4)

def load_stats() -> Dict:
    with open("stats_twitter.json", "r") as f:
        return json.load(f)

def print_stats(stats):
    print(json.dumps(stats, indent=4, sort_keys=True))

In [3]:
# Modification sur le style du graph

def highlight_nodes(graph: nx.Graph, nodes: List = []):
    reset_colors(graph)
    for node in nodes:
        graph.nodes[node]["viz"]["color"] = {'r': 255, 'g': 0, 'b': 0, 'a': 1}

def highlight_nodes_importance(graph: nx.Graph, node_importance : Dict = {}):
    reset_colors(graph)
    min = np.min(list(node_importance.values()))
    max = np.max(list(node_importance.values()))
    norm = matplotlib.colors.Normalize(vmin=min, vmax=max)
    for node, importance in node_importance.items():
        g = int(255 * norm(importance))
        r = 255 - g
        graph.nodes[node]["viz"]["color"] = {'r': r, 'g': g, 'b': 0, 'a': 1}

def highlight_nodes_communities(graph: nx.Graph, communities: List[List] = []):
    reset_colors(graph)
    cmap: LinearSegmentedColormap = plt.get_cmap('hsv')
    for i, community in enumerate(communities):
        (r, g, b, a) = cmap(i / len(communities))
        r, g, b, a = int(r * 255), int(g * 255), int(b * 255), a
        for node in community:
            graph.nodes[node]["viz"]["color"] = {'r': r, 'g': g, 'b': b, 'a': a}


def reset_colors(graph: nx.Graph):
    for node in graph:
        if(not "viz" in graph.nodes[node]):
            graph.nodes[node]["viz"] = {}
        graph.nodes[node]["viz"]["color"] = {'r': 173, 'g': 216, 'b': 230, 'a': 1}


def circular_layout(graph: nx.Graph):
    pos = nx.circular_layout(graph, scale=4*graph.number_of_nodes())
    for node, position in pos.items():
        if(not "viz" in graph.nodes[node]):
            graph.nodes[node]["viz"] = {}
        graph.nodes[node]["viz"]["position"] = {"x": position[0], "y": position[1], "z": 0.0}

In [18]:
# Parcours en largeur

def bfs_sampling(G, start_node, num_nodes_to_sample):
    visited = set()
    queue = [start_node]

    while queue and len(visited) < num_nodes_to_sample:
        node = queue.pop(0)
        if node not in visited:
            visited.add(node)
            neighbors = list(G.neighbors(node))
            random.shuffle(neighbors)
            queue.extend(neighbors)

    subgraph = G.subgraph(visited)
    return subgraph

In [19]:
DATASET_PATH = 'datasets/twitter_combined.txt'
STATS = load_stats()

G = nx.read_edgelist(path=DATASET_PATH, create_using=nx.DiGraph(), nodetype=int)

# Creation du sous graph avec 5000 noeuds. On garde une partie du graph en partant du noeud de plus grande centralité.

centrality = nx.degree_centrality(G)
max_degree_node = max(centrality, key=centrality.get)
G = bfs_sampling(G, max_degree_node,1000)


circular_layout(G)
reset_colors(G)

nx.write_gexf(G, "graphs/graph.gexf")

In [20]:
STATS["number_of_nodes"] =  G.number_of_nodes()
save_stats(STATS)

STATS["number_of_edges"] =  G.number_of_edges()
save_stats(STATS)

print("nombre de noeuds : ",STATS["number_of_nodes"])
print("nombre d'arêtes : ",STATS["number_of_edges"])

nombre de noeuds :  1000
nombre d'arêtes :  6750


## Étude de la centralité

In [21]:
# Calcul de la centralité de degré pour les nœuds échantillonnés

STATS["degree_centrality"] = nx.degree_centrality(G)
save_stats(STATS)

highlight_nodes_importance(G, STATS["degree_centrality"])
nx.write_gexf(G, "graphs/degree_centrality.gexf")

In [22]:
# Calcul de la centralité avec pagerank

STATS["pagerank"] = nx.pagerank(G)
save_stats(STATS)

highlight_nodes_importance(G, STATS["pagerank"])
nx.write_gexf(G, "graphs/pagerank.gexf")

In [23]:
# Calcul la centralité avec closeness_centrality

STATS["closeness_centrality"] = nx.closeness_centrality(G)
save_stats(STATS)
highlight_nodes_importance(G, STATS["closeness_centrality"])
nx.write_gexf(G, "graphs/closeness_centrality.gexf")

In [24]:
# Calcul la centralité avec betweenness_centrality

STATS["betweenness_centrality"] = nx.betweenness_centrality(G)
save_stats(STATS)
highlight_nodes_importance(G, STATS["betweenness_centrality"])
nx.write_gexf(G, "graphs/betweenness_centrality.gexf")

In [25]:
methodes = {"degree_centrality":STATS["degree_centrality"],"betweenness_centrality":STATS["betweenness_centrality"],
            "closeness_centrality":STATS["closeness_centrality"],"pagerank":STATS["pagerank"]}

## Independant cascade

In [26]:
# Modèle Independent_cascade pour la diffusion des informations. On prend w(u,v) = 1/dv-

def independent_cascade_simulation(G, seed_users):
    active_users = seed_users.copy()
    new_active_users = seed_users.copy()

    while new_active_users:
        newly_activated_users = []

        for active_user in new_active_users:
            for neighbor in G.neighbors(active_user):
                if neighbor not in active_users:
                    indegree = G.in_degree(neighbor)
                    p = 1 / indegree

                    if random.random() <= p:
                        newly_activated_users.append(neighbor)
                        active_users.append(neighbor)

        new_active_users = newly_activated_users

    return active_users

In [27]:
# On souhaite comparer les différentes méthodes sur la dffusion avec Independant cascade. On fait des tests avec la taille de seed_users qui varie

K_values = [5,10,20,40]
results_df = pd.DataFrame(index=K_values, columns=methodes)

num_simulations = 10

for centrality_measure, centrality_values in methodes.items():
    sorted_users = sorted(centrality_values, key=centrality_values.get, reverse=True)
    for K in K_values:
        top_K_users = sorted_users[:K]
        total_activated_users = 0
        for _ in range(num_simulations):
            activated_users = independent_cascade_simulation(G, top_K_users)
            total_activated_users += len(activated_users)

        average_activated_users = total_activated_users / num_simulations
        results_df.at[K, centrality_measure] = average_activated_users

results_df

,degree_centrality,betweenness_centrality,closeness_centrality,pagerank
5,632.5,627.3,630.6,628.0
10,656.7,637.3,627.2,640.6
20,661.6,663.3,669.0,652.0
40,680.6,684.7,679.0,680.6


In [30]:
# Influence Maximization

def influence_maximization_cascade(G, K,num_simulations=3):
    seed_users = []

    for _ in range(K):
        best_user = -1
        best_increase = 0

        for user in G.nodes:
            if user not in seed_users:
                temp_seed_users = seed_users.copy()
                temp_seed_users.append(user)

                total_activated_users = 0
                for _ in range(num_simulations):
                    activated_users = independent_cascade_simulation(G, temp_seed_users)
                    total_activated_users += len(activated_users)

                average_activated_users = total_activated_users / num_simulations
                increase = average_activated_users - len(seed_users)

                if increase > best_increase:
                    best_user = user
                    best_increase = increase

        seed_users.append(best_user)

    return seed_users

In [31]:
# Compléter result.df et ajouter la méthode influence maximization cascade

K_values = [5]
results_df['influence_maximization'] = None
num_simulations = 10

# Run the greedy influence maximization for different K values

for K in K_values:
    seed_users = influence_maximization_cascade(G, K)
    total_activated_users = 0
    for _ in range(num_simulations):
        activated_users = independent_cascade_simulation(G, seed_users)
        total_activated_users += len(activated_users)

    average_activated_users = total_activated_users / num_simulations
    results_df.at[K, 'influence_maximization'] = average_activated_users

results_df

5


,degree_centrality,betweenness_centrality,closeness_centrality,pagerank,influence_maximization
5,632.5,627.3,630.6,628.0,627.0
10,656.7,637.3,627.2,640.6,None
20,661.6,663.3,669.0,652.0,None
40,680.6,684.7,679.0,680.6,None


## Linear treshold

In [32]:
# Modèle Linear Threshold pour la diffusion des informations. On utilise un treshold aléatoire pour chaque arrete.

def linear_threshold_simulation(G, seed_users):
    active_users = seed_users.copy()
    new_active_users = seed_users.copy()

    while new_active_users:
        new_active_users_temp = []
        for user in new_active_users:
            neighbors = G.neighbors(user)
            for neighbor in neighbors:
                if neighbor not in active_users:
                    active_neighbors = [n for n in G.neighbors(neighbor) if n in active_users]
                    threshold = random.random()  # Generate a random threshold for each iteration
                    if len(active_neighbors) / G.degree(neighbor) >= threshold:
                        new_active_users_temp.append(neighbor)
                        active_users.append(neighbor)
        new_active_users = new_active_users_temp

    return active_users

In [33]:
# On souhaite comparer les différentes méthodes sur la dffusion avec Linear treshold. On fait des tests avec la taille de seed_users qui varie

K_values = [5,10,20,40]
results_df_treshold = pd.DataFrame(index=K_values, columns=methodes)

num_simulations = 10

for centrality_measure, centrality_values in methodes.items():
    sorted_users = sorted(centrality_values, key=centrality_values.get, reverse=True)
    for K in K_values:
        top_K_users = sorted_users[:K]
        total_activated_users = 0
        for _ in range(num_simulations):
            activated_users = linear_threshold_simulation(G, top_K_users)
            total_activated_users += len(activated_users)

        average_activated_users = total_activated_users / num_simulations
        results_df_treshold.at[K, centrality_measure] = average_activated_users

results_df_treshold

,degree_centrality,betweenness_centrality,closeness_centrality,pagerank
5,410.2,413.7,447.7,444.5
10,447.8,433.9,456.2,464.6
20,465.2,468.5,485.8,484.0
40,516.4,507.4,517.8,515.7


In [34]:
# Influence Maximization

def influence_maximization_threshold(G, k, num_simulations=3):
    seed_users = []

    for _ in range(k):
        best_user = None
        best_influence = 0

        for user in G.nodes():
            if user not in seed_users:
                temp_seed_users = seed_users.copy()
                temp_seed_users.append(user)

                total_influence = 0
                for _ in range(num_simulations):
                    active_users = linear_threshold_simulation(G, temp_seed_users)
                    total_influence += len(active_users)

                average_influence = total_influence / num_simulations

                if average_influence > best_influence:
                    best_user = user
                    best_influence = average_influence

        seed_users.append(best_user)

    return seed_users

In [40]:
# Compléter result.df_treshold et ajouter la méthode influence maximization treshold

K_values = [5]
results_df_treshold['influence_maximization'] = None
num_simulations = 10

# Run the greedy influence maximization for different K values

for K in K_values:
    seed_users = influence_maximization_threshold(G, K)
    
    total_activated_users = 0
    for _ in range(num_simulations):
        activated_users = linear_threshold_simulation(G, seed_users)
        total_activated_users += len(activated_users)

    average_activated_users = total_activated_users / num_simulations

KeyboardInterrupt: 

In [39]:
results_df_treshold.at[5, 'influence_maximization'] = average_activated_users
results_df_treshold

,degree_centrality,betweenness_centrality,closeness_centrality,pagerank,influence_maximization
5,410.2,413.7,447.7,444.5,401.5
10,447.8,433.9,456.2,464.6,None
20,465.2,468.5,485.8,484.0,None
40,516.4,507.4,517.8,515.7,None
